<a href="https://colab.research.google.com/github/Shreyas-13/Tensorflow-Developer/blob/main/00_Tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow Fundamentals

Outline:
* Intro to tensors
* Getting info from tensors
* Manipulating tensors
* Tensors and Numpy
* Using @tf.function(a way to speed up regular python func)
* Using GPU with tensorflow

# Intro to Tensors

### Creating tensors using `tf.constant`

In [3]:
import tensorflow as tf

print(tf.__version__)

##Creating a tensor
scalar = tf.constant(7)
scalar

2.8.0


<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [4]:
#Creating a vector

vector = tf.constant([13,2], dtype=tf.float16)
vector

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([13.,  2.], dtype=float16)>

In [5]:
vector.ndim

1

In [6]:
#Creating a matrix

matrix = tf.constant([[10,20],
                      [20,30],
                     [30,40]])
matrix

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10, 20],
       [20, 30],
       [30, 40]], dtype=int32)>

In [7]:
matrix.ndim

2

In [8]:
#Creating a 3-D tensor

tensor = tf.constant([[[10,20,30],
                       [20,30,40]],
                      [[13,2,2000],
                       [13,2,2001]],
                      [[16,11,2002],
                       [12,12,12]],
                      [[12,23,43],
                      [12,45,76]]])
tensor

<tf.Tensor: shape=(4, 2, 3), dtype=int32, numpy=
array([[[  10,   20,   30],
        [  20,   30,   40]],

       [[  13,    2, 2000],
        [  13,    2, 2001]],

       [[  16,   11, 2002],
        [  12,   12,   12]],

       [[  12,   23,   43],
        [  12,   45,   76]]], dtype=int32)>

In [9]:
tensor.ndim

3

* scalar: A variable with single value
* vector: A set of values in single dimension[a single subscript is requiured hence 1-D]
* matrix: A set of vector having 2-dimension[2 subscripts required hence 2-D e.g. arr[0][0]]
* tensor: A set of matrices having n-dimension(dimension refers to the no. of digits required to indicate a particular element in an array)

### Creating tensors with `tf.Variable`

In [10]:
##Creating a tensor using tf.Variable

changable_tensor = tf.Variable([10,23])
changable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10, 23], dtype=int32)>

In [11]:
##Updating the tensor value
changable_tensor[0].assign(13)
changable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([13, 23], dtype=int32)>

Summary:
  * tf.Variable unlike tf.constant can create tensors that can be updated when required

**Note-** It is good practice to declare your tensors as constant and change them to variable when required.

### Creating Random Tensors

Random tensors are used when we're trying to initialize the weights of a NN

In [54]:
'''Creating a random tensor with with values ranging b/w -1,1 i.e. Normal
Distribution'''
tf.random.set_seed(1)

random_1 = tf.random.Generator.from_seed(42)
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

random_1 == random_2

'''We can also use Uniform Distribution if we want a value between 0,1'''

random_3 = tf.random.Generator.from_seed(9)
random_3 = random_3.uniform(shape=(1,2))
random_3

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.8203654 , 0.70470357]], dtype=float32)>

### Shuffling the tensors

Required so that the dataset is mixedd randomly and not same type of data is repeated together.

In [26]:
import numpy as np

matrix = np.random.rand(6).reshape(3,2)

not_shuffeled = tf.constant(matrix)
not_shuffeled

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[0.72707183, 0.59029776],
       [0.15850232, 0.71818428],
       [0.41240095, 0.38722149]])>

In [37]:
'''Here even using seed doesn't guarantee same output everytime cz of global and 
operational-level seed'''

shuffeled = tf.random.shuffle(not_shuffeled, seed=1)
shuffeled

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[0.15850232, 0.71818428],
       [0.41240095, 0.38722149],
       [0.72707183, 0.59029776]])>

In [52]:
'''Here setting tf.random.set_seed() resolves the problem'''

tf.random.set_seed(1)
shuffeled = tf.random.shuffle(not_shuffeled, seed=11)
shuffeled

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[0.15850232, 0.71818428],
       [0.41240095, 0.38722149],
       [0.72707183, 0.59029776]])>

### X